In [20]:
from bs4 import BeautifulSoup
import random
# Requests
import requests
from tqdm import tqdm
import pandas as pd
import numpy as np
import sys
import psycopg2
from psycopg2 import OperationalError, errorcodes, errors
import datetime

import matplotlib.pyplot as plt
import seaborn as sns

pd.set_option('display.max_columns', None)

In [75]:
franquicias_hist = pd.read_csv('../datos_finales/franquicias_historicas.csv')
standings_2003 = pd.read_csv('../datos_finales/standings_2003.csv')
salarios_jordan = pd.read_csv('../datos_finales/salarios_jordan.csv')
df_regular_season_jordan = pd.read_csv('../datos_finales/regular_season_jordan.csv')
df_temporadas_jordan = pd.read_csv('../datos_finales/temporadas_jordan.csv')
df_playoffs_jordan = pd.read_csv('../datos_finales/playoffs_jordan.csv')
df_campeonato_jordan = pd.read_csv('../datos_finales/campeonatos_jordan.csv')
standings_2024 = pd.read_csv('../datos_finales/standings_2024.csv')
salarios_lebron = pd.read_csv('../datos_finales/salarios_lebron.csv')
df_regular_season_lebron = pd.read_csv('../datos_finales/regular_season_lebron.csv')
df_playoff_lebron = pd.read_csv('../datos_finales/playoffs_lebron.csv')
df_temporadas_lebron = pd.read_csv('../datos_finales/temporadas_lebron.csv')
df_campeonato_lebron = pd.read_csv('../datos_finales/campeonatos_lebron.csv')

In [3]:
try:
    conexion = psycopg2.connect(
    database = 'Supermercados',
    user = 'postgres',
    password = 'admin',
    host = 'localhost',
    port = '5432')
except OperationalError as e: #el alias es e
    if e.pgcode == errorcodes.INVALID_PASSWORD:
        print('la contraseña es erronea')
    elif e.pgcode == errorcodes.CONNECTION_EXCEPTION:
        print('error de conexion')
    else:
        print(f'el error es el siguiente {e}')

In [79]:
salarios_lebron.dtypes

team_code              object
Salario                 int64
Salario Actualizado     int64
Id_temporada            int64
dtype: object

In [77]:
salarios_lebron.dtypes

team_code              object
Salario                 int64
Salario Actualizado     int64
Id_temporada            int64
dtype: object

In [80]:
salarios_lebron.head(1)

,team_code,Salario,Salario Actualizado,Id_temporada
0,LAL,47607350,47607350,21
